In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('./01_raw_data/EV_project/경제 지표/기준금리.csv', encoding='cp949')

In [ ]:
df_rate = df.copy()

In [9]:
import pandas as pd

# 가정: df_rate_raw 라는 DataFrame에 아래 컬럼이 있습니다.
#   - '변경일자' (예: 2025)
#   - 'Unnamed: 1' (예: '05월 29일')
#   - '기준금리'


# 1) '변경일자' + 'Unnamed: 1' 합쳐서 datetime으로 파싱
df_rate['date'] = pd.to_datetime(
    df_rate['변경일자'].astype(str) + ' ' + df_rate['Unnamed: 1'],
    format='%Y %m월 %d일'
)

# 2) 기준금리만 남기고 정렬
df_rate = df_rate[['date', '기준금리']].sort_values('date')

# 3) 일별로 확장 — 이전 값으로 채워넣기
daily_rate = (
    df_rate
    .set_index('date')
    .resample('D')
    .ffill()
    .reset_index()
)

# 4) 월별 평균 계산
monthly_rate = (
    daily_rate
    .groupby(daily_rate['date'].dt.to_period('M'))
    .agg({'기준금리': 'mean'})
    .reset_index()
)
monthly_rate['년도'] = monthly_rate['date'].dt.year
monthly_rate['월']   = monthly_rate['date'].dt.month
monthly_rate = monthly_rate[['년도', '월', '기준금리']] \
    .rename(columns={'기준금리': '평균기준금리'})

# 5) df_merged_all 과 병합
df_merged_all = pd.merge(
    df_merged_all,
    monthly_rate,
    on=['년도', '월'],
    how='left'
)

# 6) 결과 확인
print(df_merged_all[['년도','월','평균기준금리']].drop_duplicates().head())


NameError: name 'df_merged_all' is not defined